In [1]:
import pandas as pd
import numpy as np

### 1. Постановка задачи

Имеется файл 1 с временной статистикой работы асессоров над однотипным заданием.

Задание может состоять из одного или несколько микрозаданий. Время резервирования задания (assigned_ts) указывает на тот момент, когда система назначила определенного асессора исполнителем этого задания. Этот момент может совпадать с временем начала работы асессора над заданием, а может и не совпадать (асессор может отойти выпить чаю, а потом приступить к заданию, асессор может выполнять предыдущее задание, в то время как за ним зарезервированы новые).

Предположим, что асессор за 30 секунд своего рабочего времени получает N рублей.

Какую оплату вы считаете справедливой для выполнения асессором одного микрозадания из этого файла?

### 2. Понимание данных

Формат файла: login tid Microtasks assigned_ts tclosed_ts; разделитель — табуляция \t

Пояснение к формату: 
* login — логин асессора
* tid — id оцениваемого задания (task id)
* Microtasks – количество микрозаданий в одном задании
* assigned_ts — время резервирования системой задания для асессора
* closed_ts — точное время завершения работы над заданием

В данных обнаружены следующие ошибки:
* значения tid и Microtasks имеют тип float
* значения assigned_ts и closed_ts имеют тип object

Пропущенных и нулевых значений в данных нет

In [3]:
df = pd.read_csv('one.txt', '\t')

In [8]:
df.head()

,login,tid,Microtasks,assigned_ts,closed_ts
0,login0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01
1,login0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29
2,login0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10
3,login0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56
4,login0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701827 entries, 0 to 701826
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   login        701827 non-null  object 
 1   tid          701827 non-null  float64
 2   Microtasks   701827 non-null  float64
 3   assigned_ts  701827 non-null  object 
 4   closed_ts    701827 non-null  object 
dtypes: float64(2), object(3)
memory usage: 26.8+ MB


In [12]:
for i in df.columns:
    if len(df[df[i] == 0]) != 0:
        print(f'Кол-во значений равных нулю в столбце {i}:', len(df[df[i] == 0]))
    else: print(f'Значений равных нулю в столбце {i} - нет')

Значений равных нулю в столбце login - нет
Значений равных нулю в столбце tid - нет
Значений равных нулю в столбце Microtasks - нет
Значений равных нулю в столбце assigned_ts - нет
Значений равных нулю в столбце closed_ts - нет


### 3. Подготовка данных

Для корректной обработки данных:
* значения tid и Microtasks необходимо перевести к типу int
* значения assigned_ts и closed_ts - к типу datetime

Также для удобства работы изменим название "Microtasks" на "microtasks"

In [14]:
# меняем имя столбца Microtasks

df.rename(columns={'Microtasks': 'microtasks'}, inplace=True)

In [21]:
# меняем тип данных tid, microtasks

df[['tid', 'microtasks']] = df[['tid', 'microtasks']].astype(int)

In [32]:
# меняем тип данных assigned_ts,closed_ts

df['assigned_ts'] = pd.to_datetime(df['assigned_ts'], format="%Y/%m/%d %H:%M:%S")
df['closed_ts'] = pd.to_datetime(df['closed_ts'], format="%Y/%m/%d %H:%M:%S")

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701827 entries, 0 to 701826
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   login        701827 non-null  object        
 1   tid          701827 non-null  int32         
 2   microtasks   701827 non-null  int32         
 3   assigned_ts  701827 non-null  datetime64[ns]
 4   closed_ts    701827 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int32(2), object(1)
memory usage: 21.4+ MB


### 4. Анализ данных

Для решения задачи расчитаем разницу между временем резервирования задания и временем его сдачи. Затем вычислим среднее время выполнения для задач с разным количеством микрозаданий. Определим среднее время выполнения одного микрозадания и вычислим сумму оплаты за него.

In [37]:
df.head()

,login,tid,microtasks,assigned_ts,closed_ts
0,login0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01
1,login0,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29
2,login0,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10
3,login0,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56
4,login0,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50


In [49]:
# определяем разницу между временем резервирования и временнем сдачи задачи

df['time_ts'] = df['closed_ts'] - df['assigned_ts']

In [69]:
df.head()

,login,tid,microtasks,assigned_ts,closed_ts,time_ts
0,login0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01,0 days 01:03:22
1,login0,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29,0 days 00:17:59
2,login0,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10,0 days 01:08:39
3,login0,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56,0 days 00:45:26
4,login0,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50,0 days 00:46:08


In [85]:
# переводим полученненное время выполнения заданий в секунды

s = []
for i in df['time_ts']:
    s.append(i.components[0] * 86400 +  i.components[1] * 3600 + i.components[2] * 60+ i.components[3])
    df['sec_ts'] = s

In [89]:
df.head()

,login,tid,microtasks,assigned_ts,closed_ts,time_ts,sec_ts
0,login0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01,0 days 01:03:22,3802
1,login0,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29,0 days 00:17:59,1079
2,login0,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10,0 days 01:08:39,4119
3,login0,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56,0 days 00:45:26,2726
4,login0,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50,0 days 00:46:08,2768


In [105]:
# группируем строки по количеству микрозаданий, усредняя время выполнения всего задания

task_sec = df[['microtasks','sec_ts']].groupby('microtasks').mean()
task_sec.head()

,sec_ts
microtasks,
1,552.505948
2,1065.803531
3,1641.166469
4,2263.476055
5,3084.908618


In [139]:
# вычисляем среднее время выполнения одного задания

c_microtasks = task_sec.index

time_microtasks = []
i = 0
for i in range(len(task_sec['sec_ts'])):
    time_microtasks.append(task_sec['sec_ts'].iloc[i] / c_microtasks[i])
    i += 1
    
task_sec['time_microtasks'] = time_microtasks

print('Среднее время выполнения одного микрозадания', task_sec['time_microtasks'].mean())

Среднее время выполнения одного микрозадания 369.84573452111977


### Вывод

Если считать, что за 30 секунд рабочего времени ассесор получает N рублей, то, исходя их проведенного анализа, **справедливой оплатой за выполнение одного микрозадания будет выступать (370 * N) / 30** 